# Rapport de projet de session - IFT 712
### Objectif du projet : tester six méthodes de classification sur une base de données Kaggle.

Nous avons choisi la base de données (INSERER BASE DE DONNEES) car (INSERER RAISONS).
Les méthodes de classification que nous allons tester sont les suivantes : (A MODIFIER EN FONCTION DES CHOIX DE L'EQUIPE)
* Modèle génératif
* K plus proches voisins
* Régression logistique
* Méthode à noyaux
* SVM
* Perceptron multicouches
Pour cela, nous allons utiliser la bibliothèque scikit-learn. Vous pourrez trouver notre tavail sur notre [Github][1]
[1]: https://github.com/MorganChabaudENSSAT/projet_ift712